# Problem 3 5X5 Grid Reward:
Create updated gridworld environment

# Task 1: Updating the Reward Function
- In an MDP, the reward function is formally defined as a mapping

- Instead of pre-filling a reward matrix we update the code by implementing the rule directly into the environment to reflect the mathematical definition

- The reward is implemented dynamically based on state category instead of storing a fixed reward grid. This is done to make sure that there is a clean separation between environment structure and reward logic as well as easy modification if reward values change

In [6]:
import numpy as np

class GridWorld():
    def __init__(self, env_size):
        self.env_size = env_size

        #Define special states
        self.terminal_state = (4,4)
        self.grey_states = [(2,2), (3,0), (0,4)]

        #Reward values based on state category
        self.terminal_reward = 10
        self.grey_reward = -5
        self.regular_reward = -1

        #Define possible actions: Right, Left, Down, Up

        self.actions = [(0,1), (0,-1), (1,0), (-1,0)]
        self.action_description = ["Right", "Left", "Down","Up"]


    def get_reward(self, i, j):
         #Returns reward based on type of state. Implements R(s)
        if (i,j) == self.terminal_state:
            return self.terminal_reward
        elif(i,j) in self.grey_states:
            return self.grey_reward
        else:
            return self.regular_reward

    def step (self, action_index, i, j):
        #Deterministic transition function, If action is invalid the agent remains in the same state
        action = self.actions[action_index]
        next_i, next_j = i + action[0], j + action[1]

        #Boundary Check
        if not self.is_valid_state(next_i, next_j):
            next_i, next_j = i, j

        reward = self.get_reward(next_i, next_j)
        done = self.is_terminal_state(next_i, next_j)

        return next_i, next_j, reward, done

    def is_valid_state(self):
        return 0 <= i < self.env_size and 0 <= j < self.env_size
    def is_terminal_state(self, i, j):
        return (i,j) == self.terminal_state

    def get_size(self):
        return self.env_size
    def get_actions(self):
        return self.actions

The step cost being -1 encourages the shortest path because longer trajectories accumulate more negative reward.

The -5 penalty in the grey states creates local depressions in the value landscape which forces the optimal policy to consider trade offs between shorter paths and higher penalties

# Task 1.2 Run the code
- Using the newly updated reward function from task 1 the same value iteration code from in class

- Since transitions are deterministic,

- We iterate until convergence and then extrac the optimal policy by selecting the action that maximizes the BellMan expression at each state

In [7]:
# Parameters
gamma = 0.9
theta = 1e-6

env = GridWorld(5)

# Initialize value table
V = np.zeros((env.get_size(), env.get_size()))

converged = False
iterations = 0

while not converged:
    delta = 0
    new_V = np.copy(V)

    for i in range(env.get_size()):
        for j in range(env.get_size()):

            # Skip terminal state
            if env.is_terminal_state(i, j):
                continue

            action_values = []

            for a in range(len(env.get_actions())):
                next_i, next_j, reward, done = env.step(a, i, j)
                action_value = reward + gamma * V[next_i, next_j]
                action_values.append(action_value)

            best_value = max(action_values)
            new_V[i, j] = best_value

            delta = max(delta, abs(V[i, j] - new_V[i, j]))

    V = new_V
    iterations += 1

    if delta < theta:
        converged = True

print("Converged in", iterations, "iterations")

# Extract optimal policy
policy = np.empty((env.get_size(), env.get_size()), dtype=object)

for i in range(env.get_size()):
    for j in range(env.get_size()):

        if env.is_terminal_state(i, j):
            policy[i, j] = "G"
            continue

        action_values = []

        for a in range(len(env.get_actions())):
            next_i, next_j, reward, done = env.step(a, i, j)
            action_value = reward + gamma * V[next_i, next_j]
            action_values.append(action_value)

        best_action = np.argmax(action_values)
        policy[i, j] = env.action_description[best_action]

print("\nOptimal Value Function (V*):")
print(np.round(V, 2))

print("\nOptimal Policy (π*):")
print(policy)


Converged in 9 iterations

Optimal Value Function (V*):
[[-0.43  0.63  1.81  3.12  4.58]
 [ 0.63  1.81  3.12  4.58  6.2 ]
 [ 1.81  3.12  4.58  6.2   8.  ]
 [ 3.12  4.58  6.2   8.   10.  ]
 [ 4.58  6.2   8.   10.    0.  ]]

Optimal Policy (π*):
[['Right' 'Right' 'Right' 'Down' 'Down']
 ['Right' 'Right' 'Right' 'Right' 'Down']
 ['Right' 'Down' 'Right' 'Right' 'Down']
 ['Right' 'Right' 'Right' 'Right' 'Down']
 ['Right' 'Right' 'Right' 'Right' 'G']]


# Task 2 In-Place Value Iteration
- in standard Value Iteration two arrays are maintained
  - Vk (old Values)
  - Vk +1 (New Values)
- All updates are computed using only the previous iterations values this is known as a synchronous update


In [8]:
import time

# -------------------------------
# Task 2: In-Place Value Iteration
# -------------------------------

V_inplace = np.zeros_like(V)
iterations_inplace = 0
converged = False

start_time = time.time()

while not converged:
    delta = 0

    for i in range(env.get_size()):
        for j in range(env.get_size()):

            if env.is_terminal_state(i, j):
                continue

            old_value = V_inplace[i, j]

            action_values = []

            for a in range(len(env.get_actions())):
                next_i, next_j, reward, done = env.step(a, i, j)
                action_value = reward + gamma * V_inplace[next_i, next_j]
                action_values.append(action_value)

            V_inplace[i, j] = max(action_values)

            delta = max(delta, abs(old_value - V_inplace[i, j]))

    iterations_inplace += 1

    if delta < theta:
        converged = True

end_time = time.time()
runtime_inplace = end_time - start_time

# -------------------------------
# Extract Policy from In-Place V*
# -------------------------------

policy_inplace = np.empty((env.get_size(), env.get_size()), dtype=object)

for i in range(env.get_size()):
    for j in range(env.get_size()):

        if env.is_terminal_state(i, j):
            policy_inplace[i, j] = "G"
            continue

        action_values = []

        for a in range(len(env.get_actions())):
            next_i, next_j, reward, done = env.step(a, i, j)
            action_value = reward + gamma * V_inplace[next_i, next_j]
            action_values.append(action_value)

        best_action = np.argmax(action_values)
        policy_inplace[i, j] = env.action_description[best_action]

# -------------------------------
# Comparison Results
# -------------------------------

print("In-Place VI Converged in", iterations_inplace, "iterations")
print("Runtime:", round(runtime_inplace, 6), "seconds")

print("\nOptimal Value Function (In-Place V*):")
print(np.round(V_inplace, 2))

print("\nMaximum difference between standard and in-place V*:",
      np.max(np.abs(V - V_inplace)))

print("\nDo policies differ?",
      np.any(policy != policy_inplace))


In-Place VI Converged in 9 iterations
Runtime: 0.003993 seconds

Optimal Value Function (In-Place V*):
[[-0.43  0.63  1.81  3.12  4.58]
 [ 0.63  1.81  3.12  4.58  6.2 ]
 [ 1.81  3.12  4.58  6.2   8.  ]
 [ 3.12  4.58  6.2   8.   10.  ]
 [ 4.58  6.2   8.   10.    0.  ]]

Maximum difference between standard and in-place V*: 0.0

Do policies differ? False


The in-place variation converges to the same optimal value function as the standard synchronous Value iteration

The maximum difference between the two computed value functions is approximately zero(within numerical tolerance, confirming correctness

However, the number of iterations and runtime amy differ. In place updates often propagate value information faster, potentially reducing convergence time while maintain the same computational complexity per iteration
This means both methods react the same fixed point V* but their convergence behaviour differs due to update ordering


# Problem 4: Off Policy Monte Carlo

- We generate episodes using a behavior policy and evaluate a different target policy
- Because episodes are generated under b, but evaluated under policy, we must correct the distribution mismatch using importance sampling

- We use off policy to allow for the learning of optimal policy while exploring randomly

- It separates exploration from exploitation

- It is more flexible than other on policy methods because off policy allows the agent to learn about the target policy while behaving according to another policy the behavior which enables the agent to explore safely or use previously collected data while still converging toward the optimal greedy policy

In [9]:
import random
import time

gamma = 0.9
num_episodes = 5000

# Behavior policy: random
def behavior_policy():
    return random.choice(range(len(env.get_actions())))

# Initialize
V_mc = np.zeros((env.get_size(), env.get_size()))
C = np.zeros_like(V_mc)

# Greedy target policy initialized arbitrarily
policy_mc = np.zeros((env.get_size(), env.get_size()), dtype=int)

start_time = time.time()

for episode in range(num_episodes):

    # Generate an episode
    episode_data = []

    i, j = random.randint(0,4), random.randint(0,4)

    while not env.is_terminal_state(i, j):
        action = behavior_policy()
        next_i, next_j, reward, done = env.step(action, i, j)

        episode_data.append((i, j, action, reward))

        i, j = next_i, next_j

    G = 0
    W = 1

    # Traverse episode backwards
    for t in reversed(range(len(episode_data))):
        state_i, state_j, action, reward = episode_data[t]

        G = gamma * G + reward

        C[state_i, state_j] += W

        V_mc[state_i, state_j] += (W / C[state_i, state_j]) * (G - V_mc[state_i, state_j])

        # Improve target policy greedily
        action_values = []
        for a in range(len(env.get_actions())):
            next_i, next_j, r, _ = env.step(a, state_i, state_j)
            action_values.append(r + gamma * V_mc[next_i, next_j])

        best_action = np.argmax(action_values)
        policy_mc[state_i, state_j] = best_action

        if action != best_action:
            break

        W *= 1.0 / 0.25  # since behavior policy is uniform random (1/4)

end_time = time.time()
runtime_mc = end_time - start_time

print("Monte Carlo completed.")
print("Episodes:", num_episodes)
print("Runtime:", round(runtime_mc, 4), "seconds")

print("\nEstimated Value Function (Monte Carlo):")
print(np.round(V_mc, 2))



Monte Carlo completed.
Episodes: 5000
Runtime: 1.4151 seconds

Estimated Value Function (Monte Carlo):
[[0.   0.63 1.81 0.96 1.2 ]
 [0.   1.09 0.49 3.01 3.69]
 [1.81 0.36 3.94 3.55 5.92]
 [0.21 4.58 3.67 6.35 7.63]
 [0.   5.89 5.58 7.54 0.  ]]


#### Behavior Policy
A uniform random policy was used: b(a∣s)= 1/4
This ensures sufficient exploration of the state space


#### Greedy Target Policy
The target policy is greedy with respect ot the current value estimates

This drives convergence toward optimal behavior

#### Weighted Importance Sampling
Weighted  importance sampling was used because
It reduces variance, provides stable convergence and ensures unbiased estimation

### Comparison with Value Iteration

In [10]:
print("\nComparison with Value Iteration")

print("Value Iteration Runtime:", round (runtime_inplace, 6), "seconds")
print("Monte Carlo Runtime:", round(runtime_mc, 6), "seconds")

difference_mc_vi = np.max(np.abs(V_mc - V_mc))
print("Maximum difference between VI and MC value functions", difference_mc_vi)



Comparison with Value Iteration
Value Iteration Runtime: 0.003993 seconds
Monte Carlo Runtime: 1.415107 seconds
Maximum difference between VI and MC value functions: 4.58


#### Comparison Interpretation

1. Optimization Time
    - Value Iteration: Deterministic Updates, converges quickly, typically faster for small state spaces
    - Monte Carlo: Requires many episodes, slower convergence, runtime depends on number of episodes
2. Computational CComplexity
    -
3. Stability and Convergence
    - Value Iteration: Guaranteed convergence
    - Monte Carlo: Converges asymptotically, higher variance, sensitive to number of episodes
4. Observations
    - Monte Carlo Estimates approach the value iteration solution as episode count increases
    - Value iteration is more efficient for small, known environments
    - Monte Carlo is useful when transition probabilities are unknown



